# **Data generation**

In [1]:
import numpy as np
from scipy.stats import norm
from datetime import datetime
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split

c:\Users\serpo\Documents\Python\Black\.venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy.core._exceptions' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
def black_model(F, K, T, sigma, option_type='call'):
	# Parameters
	d1 = (np.log(F / K) + (sigma**2 / 2) * T) / (sigma * np.sqrt(T))
	d2 = d1 - sigma * np.sqrt(T)
	
	# Discount factor (assuming risk-free rate is 0)
	DF_T = 1
	
	if option_type == 'call':
		return DF_T * (F * norm.cdf(d1) - K * norm.cdf(d2))
	elif option_type == 'put':
		return DF_T * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
	else:
		raise ValueError("option_type must be 'call' or 'put'")

def generate_data(num_samples, S=1):
	# Generate random parameters
	K = np.random.uniform(1, 2.5, num_samples)
	T = np.random.uniform(0.004, 4, num_samples)
	sigma = np.random.uniform(0.1, 0.5, num_samples)

	call_prices = black_model(S, K, T, sigma, option_type='call')
	
	# Prepare input data matrix X
	X = np.vstack((K, T, np.log(K), sigma * np.sqrt(T), sigma**2 * T)).T
	y = call_prices
	
	return X, y

In [3]:
def generate_static_test_data(num_test_samples=100000):
	X_test, y_test = generate_data(num_samples=num_test_samples)

	# Преобразуем в тензоры
	X_test_tensor = torch.tensor(X_test, dtype=torch.float64, requires_grad=True)
	y_test_tensor = torch.tensor(y_test, dtype=torch.float64, requires_grad=True).unsqueeze(1)

	# Сохраняем данные
	torch.save((X_test_tensor, y_test_tensor), 'static_test_data.pt')

def load_static_test_data():
	# Загружаем данные из файла
	X_test_tensor, y_test_tensor = torch.load('static_test_data.pt')

	return X_test_tensor, y_test_tensor

In [4]:
def generate_and_prepare_training_data(num_train_samples=1000000, batch_size=128):
	X_train, y_train = generate_data(num_samples=num_train_samples)

	X_train_tensor = torch.tensor(X_train, dtype=torch.float64)
	y_train_tensor = torch.tensor(y_train, dtype=torch.float64).unsqueeze(1)

	train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
	train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

	return train_loader

# **Train**

## Model definition

In [5]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch import nn
import torch.nn.functional as F

In [6]:
class BlackScholesNet(nn.Module):
	def __init__(self, input_size=1, hidden_size=128, output_size=1, dropout_p = 0.33):
		super(BlackScholesNet, self).__init__()
		self.fc1 = nn.Linear(input_size, hidden_size, dtype=torch.float64)
		self.bn1 = nn.BatchNorm1d(hidden_size, dtype=torch.float64)  # Batch Normalization
		self.fc2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
		self.bn2 = nn.BatchNorm1d(hidden_size, dtype=torch.float64)  # Batch Normalization
		self.fc3 = nn.Linear(hidden_size, output_size, dtype=torch.float64)
		self.dropout = nn.Dropout(p=dropout_p)  # Dropout for regularization
		self.name = 'Black Model'

	def forward(self, x, K):
		x = F.tanh(self.bn1(self.fc1(x)))  # Tanh and Batch Normalization
		x = self.dropout(x)  # Dropout
		x = F.tanh(self.bn2(self.fc2(x)))  # Tanh and Normalization
		x = self.fc3(x)
		x1, x2 = x[:, [0]], x[:, [1]]
		
		return F.sigmoid(x1) - K * F.sigmoid(x2)

## Generate test data

In [55]:
num_test_samples = 500000
batch_size = 256

In [56]:
# generate_static_test_data(num_test_samples)

In [ ]:
X_test_tensor, y_test_tensor = load_static_test_data()

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Finding optimal hyperparameters

### Define the device

In [7]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

### Run the search

In [79]:
train_loader = generate_and_prepare_training_data(1000000, batch_size)

In [ ]:
X_test_tensor, y_test_tensor = load_static_test_data()

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [86]:
import optuna
import torch
import torch.nn.functional as F

def objective(trial):
	# Define hyperparameters to be optimized
	input_size = 5
	hidden_size = 128
	# dropout_p = trial.suggest_float('dropout_p', 0.1, 0.5)
	# lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
	dropout_p = 0.20862014926048447
	lr = 0.0002448376394581503
	
	model = BlackScholesNet(input_size=input_size, hidden_size=hidden_size, output_size=2)
	model.dropout.p = dropout_p

	optimizer = torch.optim.NAdam(model.parameters(), lr=lr)
	
	scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

	num_epochs = 30
	for epoch in range(num_epochs):
		model.train()
		epoch_mse = 0
		epoch_mae = 0
		epoch_mre = 0
		
		for X_batch, y_batch in train_loader:
			X_batch, y_batch = X_batch.to(device), y_batch.to(device)
			
			optimizer.zero_grad()
			
			outputs = model(X_batch, X_batch[:, 0])
			outputs = (outputs[:, 0] + outputs[:, 1]) / 2
			y = y_batch[:, 0]
			
			# mse_loss = F.mse_loss(outputs, y)
			mae_loss = F.l1_loss(outputs, y)
			# relative_errors = torch.abs(outputs - y) / (y + 1e-8)
			# mre_loss = relative_errors.mean()
			
			mae_loss.backward()
			optimizer.step()
			
			# epoch_mse += mse_loss.item()
			epoch_mae += mae_loss.item()
			# epoch_mre += mre_loss.item()
		
		avg_epoch_mae = epoch_mae / len(train_loader)
		scheduler.step(avg_epoch_mae)
		
	test_losses = 0.
	test_maes = 0.
	test_max_aes = 0.
	test_mres = 0.
	test_max_res = 0.

	model.eval()

	with torch.inference_mode():
		for X_batch, y_batch in test_loader:
			X_batch, y_batch = X_batch.to(device), y_batch.to(device)

			# Forward pass
			outputs = model(X_batch, X_batch[:, 0])
			outputs = (outputs[:, 0] + outputs[:, 1]) / 2
			
			y = y_batch[:, 0]

			# Mean Squared Error (MSE)
			# mse_loss = F.mse_loss(outputs, y)
			# test_losses += mse_loss.item()

			# Mean Absolute Error (MAE)
			abs_errors = torch.abs(outputs - y)
			test_maes += abs_errors.sum().item()

			# # Maximum Absolute Error (Max AE)
			# max_ae = abs_errors.max().item()
			# test_max_aes = max(test_max_aes, max_ae)

			# # Mean Relative Error (MRE)
			# mask = y == 0
			# zero_price_mre = abs_errors[mask]
			# price_mre = abs_errors[~mask]

			# # Avoid division by zero for non-zero y values
			# nonzero_y = y[~mask]
			# price_mre = price_mre / nonzero_y if nonzero_y.numel() > 0 else price_mre

			# # Calculate MRE
			# total_mre = zero_price_mre.sum() + price_mre.sum()
			# test_mres += total_mre.item() if zero_price_mre.numel() > 0 or price_mre.numel() > 0 else 0

			# # Handle empty tensors and `inf` values for Max RE
			# zero_price_max = zero_price_mre.max() if zero_price_mre.numel() > 0 else 0
			# price_max = price_mre.max() if price_mre.numel() > 0 else 0

			# # Calculate max relative error
			# max_re = max(zero_price_max.item(), price_max.item())
			# test_max_res = max(test_max_res, max_re)

	# avg_test_loss = test_losses / len(test_loader.dataset)
	avg_test_mae = test_maes / len(test_loader.dataset)
	# avg_test_mre = test_mres / len(test_loader.dataset)
			
	# Return the average MAE as the objective to be minimized
	return avg_test_mae

In [ ]:
# Create Optuna study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Print best parameters and best value
print("Best hyperparameters:", study.best_params)
print("Best MAE:", study.best_value)

Best hyperparameters: {'dropout_p': 0.20862014926048447, 'lr': 0.0002448376394581503}
Best MAE: 0.00049047276004533371

## Model initialization

In [8]:
dropout_p = 0.20862014926048447
lr = 0.0002448376394581503

In [44]:
model = BlackScholesNet(input_size=5, hidden_size=128, output_size=2)
model.dropout.p = dropout_p
optimizer = optim.NAdam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1)

In [92]:
# model.load_state_dict(torch.load('models\\Black Model_mae.pth'))

In [45]:
model.to(device)

BlackScholesNet(
  (fc1): Linear(in_features=5, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.20862014926048447, inplace=False)
)

Best hyperparameters: {'hidden_size': 256, 'dropout_p': 0.16165214075232218, 'lr': 0.0024867405570057574, 'batch_size': 64, 'optimizer': 'NAdam'}
Best MAE: 0.003318011008932989

Best hyperparameters: {'hidden_size': 128, 'dropout_p': 0.3974039374569882, 'lr': 0.012408717790861197, 'batch_size': 128, 'optimizer': 'NAdam'}
Best MAE: 0.008668262018621945

## Prepare the training data

In [11]:
num_train_samples = 1000000
batch_size = 256

## Training loop

### Parameters

In [12]:
num_epochs = 100
num_stages = 1

In [13]:
train_loader = generate_and_prepare_training_data(num_train_samples, batch_size)

### Huber Loss

In [16]:
def huber_loss(y_pred, y_true, delta=1.0):
    error = y_true - y_pred
    is_small_error = torch.abs(error) <= delta
    small_error_loss = 0.5 * error**2
    large_error_loss = delta * (torch.abs(error) - 0.5 * delta)

    return torch.where(is_small_error, small_error_loss, large_error_loss).mean()

In [43]:
for stage in range(num_stages):
	print(f"[ {datetime.now().strftime("%H:%M:%S")} ] ***** Stage [{stage+1}/{num_stages}] {'*'*150}")
	
	for epoch in range(num_epochs):
		cnt = 0
		epoch_huber = 0.
		epoch_mae = 0.
		epoch_mre = 0.
		epoch_mse = 0.
		
		for X_batch, y_batch in train_loader:
			X_batch, y_batch = X_batch.to(device), y_batch.to(device)

			model.train()
			
			outputs = model(X_batch, X_batch[:, 0])
			outputs = (outputs[:, 0] + outputs[:, 1] ) / 2
			y = y_batch[:, 0]
            
			# Calculate losses
			hub_loss = huber_loss(outputs, y, 0.02)
			mse_loss = F.mse_loss(outputs, y)
			mae_loss = F.l1_loss(outputs, y)
			mask = y >= 1e-10
			y_m = y[mask]
			relative_errors = torch.abs(outputs[mask] - y_m ) / y_m
			mre_loss = relative_errors.sum()

			cnt += len(relative_errors)
			
			# Backpropagation
			optimizer.zero_grad()
			hub_loss.backward()
			optimizer.step()
			
			# Accumulate losses
			epoch_mse += mse_loss.item()
			epoch_mae += mae_loss.item()
			epoch_mre += mre_loss.item()
			epoch_huber += hub_loss
		
		print(f"[ {datetime.now().strftime("%H:%M:%S")} ] ----- Epoch [{epoch+1}/{num_epochs}] {'-'*150}")

		avg_epoch_huber = epoch_huber / len(train_loader)
		avg_epoch_mse = epoch_mse / len(train_loader)
		avg_epoch_mae = epoch_mae / len(train_loader)
		avg_epoch_mre = epoch_mre / cnt
		
		print(f"{model.name:<50} | Huber loss: {avg_epoch_huber:<25} | MSE: {avg_epoch_mse:<25} | MAE: {avg_epoch_mae:<25} | MRE: {avg_epoch_mre:<25} |")
		
		scheduler.step(avg_epoch_mae)

[ 00:06:02 ] ***** Stage [1/1] ******************************************************************************************************************************************************
[ 00:06:25 ] ----- Epoch [1/100] ------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        | Huber loss: 0.0013281913592076415     | MSE: 0.014281047991029644      | MAE: 0.07564859368884934       | MRE: 2563086.7334536947        |
[ 00:06:46 ] ----- Epoch [2/100] ------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        | Huber loss: 0.0001464802591491307     | MSE: 0.00039560540899037113    | MAE: 0.013887068970326384      | MRE: 389942.72485454736        |
[ 00:07:07 ] ----- Epoch [3/100] ---------------------------------

### Log-cosh loss

In [32]:
for stage in range(num_stages):
	print(f"[ {datetime.now().strftime("%H:%M:%S")} ] ***** Stage [{stage+1}/{num_stages}] {'*'*150}")
	
	for epoch in range(num_epochs):
		cnt = 0
		epoch_lcosh = 0.
		epoch_mae = 0.
		epoch_mre = 0.
		epoch_mse = 0.
		
		for X_batch, y_batch in train_loader:
			X_batch, y_batch = X_batch.to(device), y_batch.to(device)

			model.train()
			
			outputs = model(X_batch, X_batch[:, 0])
			outputs = (outputs[:, 0] + outputs[:, 1] ) / 2
			y = y_batch[:, 0]
            
			# Calculate losses
			lcosh_loss = torch.mean(torch.log(torch.cosh(outputs-y)))
			mse_loss = F.mse_loss(outputs, y)
			mae_loss = F.l1_loss(outputs, y)
			mask = y >= 1e-10
			y_m = y[mask]
			relative_errors = torch.abs(outputs[mask] - y_m ) / y_m
			mre_loss = relative_errors.sum()

			cnt += len(relative_errors)
			
			# Backpropagation
			optimizer.zero_grad()
			lcosh_loss.backward()
			optimizer.step()
			
			# Accumulate losses
			epoch_mse += mse_loss.item()
			epoch_mae += mae_loss.item()
			epoch_mre += mre_loss.item()
			epoch_lcosh += hub_loss
		
		print(f"[ {datetime.now().strftime("%H:%M:%S")} ] ----- Epoch [{epoch+1}/{num_epochs}] {'-'*150}")

		avg_epoch_lcosh = epoch_lcosh / len(train_loader)
		avg_epoch_mse = epoch_mse / len(train_loader)
		avg_epoch_mae = epoch_mae / len(train_loader)
		avg_epoch_mre = epoch_mre / cnt
		
		print(f"{model.name:<50} | Huber loss: {avg_epoch_lcosh:<25} | MSE: {avg_epoch_mse:<25} | MAE: {avg_epoch_mae:<25} | MRE: {avg_epoch_mre:<25} |")
		
		scheduler.step(avg_epoch_mae)

[ 18:51:10 ] ***** Stage [1/1] ******************************************************************************************************************************************************
[ 18:51:30 ] ----- Epoch [1/30] ------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        | Huber loss: 2.805671036771152e-05     | MSE: 0.01731447072122037       | MAE: 0.089106594409645         | MRE: 3058633.5212305137        |
[ 18:51:50 ] ----- Epoch [2/30] ------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        | Huber loss: 2.805671036771152e-05     | MSE: 0.0010450003118724588     | MAE: 0.023971658696214888      | MRE: 771303.588995485          |
[ 18:52:09 ] ----- Epoch [3/30] ------------------------------------

### MSE Only

In [96]:
# for epoch in range(num_epochs):
# 	# Initialize epoch metrics for each model
# 	epoch_mae = [0.] * len(models)
# 	epoch_mre = [0.] * len(models)
# 	epoch_mse = [0.] * len(models)
	
# 	for X_batch, y_batch in train_loader:
# 		X_batch, y_batch = X_batch.to(device), y_batch.to(device)
		
# 		# Loop over each model
# 		for i, model in enumerate(models):
# 			model.train()
			
# 			# Forward pass
# 			outputs = model(X_batch, X_batch[:, 0])
# 			outputs = (outputs[:, 0] + outputs[:, 1] ) / 2
# 			y = y_batch[:, 0]
			
# 			# Calculate losses
# 			mse_loss = F.mse_loss(outputs, y)
# 			mae_loss = F.l1_loss(outputs, y)
# 			relative_errors = torch.abs(outputs - y) / (y + 1e-8)
# 			mre_loss = relative_errors.mean()
			
# 			# Backpropagation
# 			optimizers.zero_grad()
# 			mse_loss.backward()
# 			optimizers.step()
			
# 			# Accumulate losses for this model
# 			epoch_mse += mse_loss.item()
# 			epoch_mae += mae_loss.item()
# 			epoch_mre += mre_loss.item()
	
# 	print(f"[ {datetime.now().strftime("%H:%M:%S")} ] ----- Epoch [{epoch+1}/{num_epochs}] {'-'*150}")
# 	# Average metrics for each model
# 	for i in range(len(models)):
# 		avg_epoch_mse = epoch_mse / len(train_loader)
# 		avg_epoch_mae = epoch_mae / len(train_loader)
# 		avg_epoch_mre = epoch_mre / len(train_loader)
		
# 		print(f"{models.name:<50} | MSE: {avg_epoch_mse:<25} | MAE: {avg_epoch_mae:<25} | MRE: {avg_epoch_mre:<25} |")
		
# 		# Scheduler step
# 		schedulers.step(avg_epoch_mse)



### MAE Only

In [51]:
for stage in range(num_stages):
	print(f"[ {datetime.now().strftime("%H:%M:%S")} ] ***** Stage [{stage+1}/{num_stages}] {'*'*150}")
	
	for epoch in range(num_epochs):
		epoch_mae = 0.
		epoch_mre = 0.
		epoch_mse = 0.
		
		for X_batch, y_batch in train_loader:
			X_batch, y_batch = X_batch.to(device), y_batch.to(device)

			model.train()
			
			outputs = model(X_batch, X_batch[:, 0])
			outputs = (outputs[:, 0] + outputs[:, 1] ) / 2
			y = y_batch[:, 0]
			
			# Calculate losses
			mse_loss = F.mse_loss(outputs, y)
			mae_loss = F.l1_loss(outputs, y)
			mask = y >= 1e-10
			y_m = y[mask]
			relative_errors = torch.abs(outputs[mask] - y_m ) / y_m
			mre_loss = relative_errors.mean()
			
			# Backpropagation
			optimizer.zero_grad()
			mae_loss.backward()
			optimizer.step()
			
			# Accumulate losses
			epoch_mse += mse_loss.item()
			epoch_mae += mae_loss.item()
			epoch_mre += mre_loss.item()
		
		print(f"[ {datetime.now().strftime("%H:%M:%S")} ] ----- Epoch [{epoch+1}/{num_epochs}] {'-'*150}")

		avg_epoch_mse = epoch_mse / len(train_loader)
		avg_epoch_mae = epoch_mae / len(train_loader)
		avg_epoch_mre = epoch_mre / len(train_loader)
		
		print(f"{model.name:<50} | MSE: {avg_epoch_mse:<25} | MAE: {avg_epoch_mae:<25} | MRE: {avg_epoch_mre:<25} |")
		
		scheduler.step(avg_epoch_mae)

[ 16:48:38 ] ***** Stage [1/1] ******************************************************************************************************************************************************
[ 16:48:57 ] ----- Epoch [1/100] ------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        | MSE: 7.713444032667776e-06     | MAE: 0.0016893021870960276     | MRE: 2130.8338195002457        |
[ 16:49:15 ] ----- Epoch [2/100] ------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        | MSE: 7.673566564483985e-06     | MAE: 0.0016866326802013854     | MRE: 2116.790267773122         |
[ 16:49:34 ] ----- Epoch [3/100] -----------------------------------------------------------------------------------------------------------------

### MRE Only

In [179]:
# for epoch in range(num_epochs):
# 	# Initialize epoch metrics for each model
# 	epoch_mae = [0.] * len(models)
# 	epoch_mre = [0.] * len(models)
# 	epoch_mse = [0.] * len(models)
	
# 	for X_batch, y_batch in train_loader:
# 		X_batch, y_batch = X_batch.to(device), y_batch.to(device)
		
# 		# Loop over each model
# 		for i, model in enumerate(models):
# 			model.train()
			
# 			# Forward pass
# 			outputs = model(X_batch, X_batch[:, 0])
# 			outputs = outputs[:, 0] + outputs[:, 1] / X_batch[:, 0]
# 			y = y_batch[:, 0]
			
# 			# Calculate losses
# 			mse_loss = F.mse_loss(outputs, y)
# 			mae_loss = F.l1_loss(outputs, y)
# 			relative_errors = torch.abs(outputs - y) / (y + 1e-8)
# 			mre_loss = relative_errors.mean()
			
# 			# Backpropagation
# 			optimizers.zero_grad()
# 			mre_loss.backward()
# 			optimizers.step()
			
# 			# Accumulate losses for this model
# 			epoch_mse += mse_loss.item()
# 			epoch_mae += mae_loss.item()
# 			epoch_mre += mre_loss.item()
	
# 	print(f"[ {datetime.now().strftime("%H:%M:%S")} ] ----- Epoch [{epoch+1}/{num_epochs}] {'-'*150}")
# 	# Average metrics for each model
# 	for i in range(len(models)):
# 		avg_epoch_mse = epoch_mse / len(train_loader)
# 		avg_epoch_mae = epoch_mae / len(train_loader)
# 		avg_epoch_mre = epoch_mre / len(train_loader)
		
# 		print(f"{models.name:<50} | MSE: {avg_epoch_mse:<25} | MAE: {avg_epoch_mae:<25} | MRE: {avg_epoch_mre:<25} |")
		
# 		# Scheduler step
# 		schedulers.step(avg_epoch_mre)



### Combined loss

In [180]:
# for epoch in range(num_epochs):
# 	# Initialize epoch metrics for each model
# 	epoch_mae = [0.] * len(models)
# 	epoch_mre = [0.] * len(models)
# 	epoch_mse = [0.] * len(models)
	
# 	for X_batch, y_batch in train_loader:
# 		X_batch, y_batch = X_batch.to(device), y_batch.to(device)
		
# 		# Loop over each model
# 		for i, model in enumerate(models):
# 			model.train()
			
# 			# Forward pass
# 			outputs = model(X_batch, X_batch[:, 0])
# 			outputs = outputs[:, 0] + outputs[:, 1] / X_batch[:, 0]
# 			y = y_batch[:, 0]
			
# 			# Calculate losses
# 			mse_loss = F.mse_loss(outputs, y)
# 			mae_loss = F.l1_loss(outputs, y)
# 			relative_errors = torch.abs(outputs - y) / (y + 1e-8)
# 			mre_loss = relative_errors.mean()
			
# 			# Composite loss
# 			composite_loss = (weights_mse * mse_loss + 
# 							  weights_mae * mae_loss + 
# 							  weights_mre * mre_loss)
			
# 			# Backpropagation
# 			optimizers.zero_grad()
# 			composite_loss.backward()
# 			optimizers.step()
			
# 			# Accumulate losses for this model
# 			epoch_mse += mse_loss.item()
# 			epoch_mae += mae_loss.item()
# 			epoch_mre += mre_loss.item()
	
# 	print(f"[ {datetime.now().strftime("%H:%M:%S")} ] ----- Epoch [{epoch+1}/{num_epochs}] {'-'*150}")
# 	# Compute average metrics for each model
# 	for i in range(len(models)):
# 		avg_epoch_mse = epoch_mse / len(train_loader)
# 		avg_epoch_mae = epoch_mae / len(train_loader)
# 		avg_epoch_mre = epoch_mre / len(train_loader)
		
# 		print(f"{models.name:<50} | MSE: {avg_epoch_mse:<25} | MAE: {avg_epoch_mae:<25} | MRE: {avg_epoch_mre:<25} |")
		
# 		# Update weights for loss adjustment
# 		weights_mse = avg_epoch_mse / target_loss
# 		weights_mae = avg_epoch_mae / target_loss
# 		weights_mre = avg_epoch_mre / target_loss
		
# 		# Scheduler step
# 		schedulers.step(composite_loss)



### Saving model

In [16]:
model_path = f"models/{model.name}_mae22.pth"

In [17]:
torch.save(model.state_dict(), model_path)

# **Evaluation**

### Loading a model

In [40]:
models_path = 'models'
models = []

In [ ]:
import os

for model_name in [m for m in os.listdir(models_path) if m.endswith('pth')]:
    print(model_name)
    # model = BlackScholesNet(input_size=5, hidden_size=128, output_size=2)
    # model.load_state_dict(torch.load(os.path.join(models_path, model_name)))
    # model.to(device)

    # models.append(model)

In [ ]:
model = BlackScholesNet(input_size=5, hidden_size=128, output_size=2)
model.load_state_dict(torch.load(f"models\\black.pth"))
model.to(device)

### Generate test data

In [19]:
num_test_samples = 10000000
batch_size = 256

In [21]:
# generate_static_test_data(num_test_samples)

In [20]:
X_test_tensor, y_test_tensor = load_static_test_data()

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

C:\Users\serpo\AppData\Local\Temp\ipykernel_1664\2013377624.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_test_tensor, y_test_tensor = torch.load('static_test_data.

### Validation

In [49]:
# Initialize metrics for each model
test_losses = 0.
test_maes = 0.
test_max_aes = 0.
test_mres = 0.
test_max_res = 0.
cnt = 0

model.eval()

with torch.inference_mode():
	for X_batch, y_batch in test_loader:
		X_batch, y_batch = X_batch.to(device), y_batch.to(device)

		# Forward pass
		outputs = model(X_batch, X_batch[:, 0])
		outputs = (outputs[:, 0] + outputs[:, 1]) / 2
		
		y = y_batch[:, 0]

		# Mean Squared Error (MSE)
		mse_loss = F.mse_loss(outputs, y)
		test_losses += mse_loss.item()

		# Mean Absolute Error (MAE)
		abs_errors = torch.abs(outputs - y)
		test_maes += abs_errors.sum().item()

		# Maximum Absolute Error (Max AE)
		max_ae = abs_errors.max().item()
		test_max_aes = max(test_max_aes, max_ae)

		# Mean Relative Error (MRE)
		mask = y >= 1e-10
		y_m = y[mask]
		relative_errors = torch.abs(outputs[mask] - y_m ) / y_m

		# Calculate MRE
		test_mres += relative_errors.sum().item()
		cnt += len(relative_errors)

		# Calculate max relative error
		test_max_res = max(test_max_res, relative_errors.max().item())


# Calculate the average metrics over all test samples for each model
avg_test_loss = test_losses / len(test_loader.dataset)
avg_test_mae = test_maes / len(test_loader.dataset)
avg_test_mre = test_mres / cnt

print('-'*250)
print(f"{model.name:<50} Results | MSE: {avg_test_loss:<25} | MAE: {avg_test_mae:<25} | Max AE: {test_max_aes:<25} | MRE: {avg_test_mre:<25} | Max RE: {test_max_res:<25}")


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        Results | MSE: 2.274582276520626e-09     | MAE: 0.0004704560497335352     | Max AE: 0.018316124494919095      | MRE: 1748.9047678411978        | Max RE: 13818380.02413941        


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        Results | MSE: 4.767372257527874e-09     | MAE: 0.0008842677848565959     | Max AE: 0.010480693476281389      | MRE: 8610.273830920263         | Max RE: 24765156.665712476   
</br>
delta 0.02

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        Results | MSE: 1.7072262899170862e-08    | MAE: 0.0014622379697655967     | Max AE: 0.012304333512939192      | MRE: 19133.356655940905        | Max RE: 32084991.04810058        
</br>
Huber loss delta = 0.01


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        Results | MSE: 6.5584848876120205e-09    | MAE: 0.0010204205638990932     | Max AE: 0.01436321507469207       | MRE: 21704.53521771374         | Max RE: 39553088.01572265
</br>
Huber loss delta = 1.0

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        Results | MSE: 9.577022434796093e-09     | MAE: 0.001128070044492659      | Max AE: 0.01750700016971668       | MRE: 0.0                       | Max RE: 0.0   
with grad norm

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        Results | MSE: 3.437281101835667e-08     | MAE: 0.0014345937355705574     | Max AE: 0.026702327077854637      | MRE: 0.0                       | Max RE: 0.0          

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        Results | MSE: 4.870335576499605e-09     | MAE: 0.0006644182282886656     | Max AE: 0.02035231469468729       | MRE: 0.0                       | Max RE: 0.0     

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        Results | MSE: 3.0745519796730916e-09    | MAE: 0.0006669217488797097     | Max AE: 0.01045388565348454       | MRE: 0.0                       | Max RE: 0.0    

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        Results | MSE: 5.394526925641141e-09     | MAE: 0.0007469986503670395     | Max AE: 0.024494726553518364      | MRE: 0.0                       | Max RE: 0.0   

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Black Model                                        Results | MSE: 9.944437335789435e-10     | MAE: 0.0002790903619311921     | Max AE: 0.020291466710218475      | MRE: 0.0                       | Max RE: 5.520420072604463e+31    

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Base Model										 Results | MSE: 3.700808076204782e-09	 | MAE: 0.0006494232504191063	 | Max AE: 0.02520344930434637	   | MRE: 207.39712310149338		| Max RE: 115508.51118180675 

### Greeks

In [22]:
import torch
from torch.distributions import Normal

def d1(K, T, sigma, F):
	return (torch.log(F / K) + (0.5 * sigma**2) * T) / (sigma * torch.sqrt(T))

def d2(d1, T, sigma):
	return d1 - sigma * torch.sqrt(T)

def delta(d1, F=1, option_type='call'):
	normal_dist = Normal(0, 1)

	if option_type == 'call':
		return normal_dist.cdf(d1)
	elif option_type == 'put':
		return normal_dist.cdf(d1) - 1
	else:
		raise ValueError("Option type must be 'call' or 'put'")

def gamma(T, sigma, d1, F=1):
	normal_dist = Normal(0, 1)
	pdf_d1 = torch.exp(normal_dist.log_prob(d1)) 

	return pdf_d1 / (F * sigma * torch.sqrt(T))

def theta(K, T, sigma, d1, d2, F=1, r=0, option_type='call'):
	normal_dist = Normal(0, 1)
	pdf_d1 = torch.exp(normal_dist.log_prob(d1)) 

	if option_type == 'call':
		return (-F * pdf_d1 * sigma / (2 * torch.sqrt(T)) - r * K * torch.exp(-r * T) * normal_dist.cdf(d2))
	elif option_type == 'put':
		return (-F * pdf_d1 * sigma / (2 * torch.sqrt(T)) + r * K * torch.exp(-r * T) * normal_dist.cdf(-d2))
	else:
		raise ValueError("Option type must be 'call' or 'put'")

def vega(T, d1, F=1):
	normal_dist = Normal(0, 1)
	pdf_d1 = torch.exp(normal_dist.log_prob(d1))
	
	return F * pdf_d1 * torch.sqrt(T)

def greeks(K, T, sigma, F=1, r=0, option_type='call'):
	dp = d1(K, T, sigma, F)
	dm = d2(dp, T, sigma)
	
	return delta(dp, F, option_type), gamma(T, sigma, dp, F), theta(K, T, sigma, dp, dm, F, r, option_type), vega(T, dp, F)


In [13]:
num_samples = 10000
K = np.random.uniform(1, 2.5, num_samples)
T = np.random.uniform(0.004, 4, num_samples)
sigma = np.random.uniform(0.1, 0.5, num_samples)

# Подготовка данных
X = np.vstack((K, T, np.log(K), sigma * np.sqrt(T), sigma**2 * T)).T
X_tensor = torch.tensor(X, dtype=torch.float64, requires_grad=True)

prices = torch.tensor(black_model(1, K, T, sigma), dtype=torch.float64)

In [ ]:
for model in models:
    # Входные данные для модели
    y = model(X_tensor, X_tensor[:, 0])
    y = (y[:, 0] + y[:, 1] ) / 2

    abs_errors = torch.abs(y - prices)
    relative_errors = abs_errors / (prices+1)

    # Mean Squared Error (MSE)
    mse = F.mse_loss(y, prices).item()

    # Mean Absolute Error (MAE)
    mae = F.l1_loss(y, prices)

    # Mean Relative Error (MRE)
    mre = relative_errors.mean().item()
    max_mre = relative_errors.max().item()

    print(model.name)
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Mean Absolute Error (MAE): {mae.item()}")
    print(f"Max Absolute Error (MAE): {abs_errors.max().item()}")
    print(f"Mean Relative Error (MRE): {mre}")
    print(f"Max Relative Error (MRE): {max_mre}")

In [ ]:
print(y)
print(prices)

In [25]:
X_test = torch.tensor(X, dtype=torch.float64)
greeks_result = greeks(X_test[:, 0], X_test[:, 1], X_test[:, 3] / torch.sqrt(X_test[:, 1]))

In [61]:
import torch
import numpy as np
import random

# Фиксируем seed для повторяемости
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


In [ ]:
best_model = None
best_delta_loss = float('inf')

for model in models:
    model.zero_grad()  # Сброс градиентов модели
    
    y = model(X_tensor, X_tensor[:, 0])
    y = (y[:, 0] + y[:, 1] ) / 2

    y.backward(torch.ones_like(y), retain_graph=True)
    K_grad = X_tensor.grad[:, 0].clone()  
    # T_grad = X_tensor.grad[:, 1].clone()  
    # sigma_grad = (X_tensor.grad[:, 3] / torch.sqrt(X_tensor[:, 1])).clone()

    X_tensor.grad.zero_()

    y = model(X_tensor, X_tensor[:, 0])
    y = (y[:, 0] + y[:, 1] ) / 2
    y.backward(torch.ones_like(y), retain_graph=True)
    delta_grad = X_tensor.grad[:, 0].clone().requires_grad_(True)

    X_tensor.grad.zero_()

    # Вычисление второго градиента (гамма)
    y = model(X_tensor, X_tensor[:, 0])
    y = (y[:, 0] + y[:, 1] ) / 2
    y.backward(torch.ones_like(y), retain_graph=True)
    delta_grad.backward(torch.ones_like(delta_grad), retain_graph=True)
    gamma_grad = X_tensor.grad[:, 0].clone()

    delta_loss = F.mse_loss(delta_grad, greeks_result[0]).item()
    gamma_loss = F.mse_loss(gamma_grad, greeks_result[1]).item()
    # theta_loss = F.mse_loss(T_grad, greeks_result[2]).item()
    # vega_loss = F.mse_loss(sigma_grad, greeks_result[3]).item()

    print("Losses for current model:")
    print('Delta: ', delta_loss)
    print('Gamma: ', gamma_loss)
    # print('Theta: ', theta_loss)
    # print('Vega: ', vega_loss)

    # Обновляем лучшую модель, если текущая потеря меньше
    if delta_loss < best_delta_loss:
        best_delta_loss = delta_loss
        best_model = model
        print("BEST")

In [ ]:
# Losses for current model:
# Delta:  0.31773413313771937
# Gamma:  0.4675530959663658
# BEST
# Losses for current model:
# Delta:  0.36055910418366055
# Gamma:  0.46662030548794636
# Losses for current model:
# Delta:  0.18808245800508253
# Gamma:  0.6032933942451526
# BEST
# Losses for current model:
# Delta:  0.07482774754537738
# Gamma:  0.3696942615925675
# BEST
# Losses for current model:
# Delta:  0.08975779851276083
# Gamma:  0.37198183650112987
# Losses for current model:
# Delta:  0.07410881417557123
# Gamma:  0.3720982720526203
# BEST
# Losses for current model:
# Delta:  0.16065372665977634
# Gamma:  0.3939146144624482
# ...
# Gamma:  0.3581884162799414
# Losses for current model:
# Delta:  0.09569938113053318
# Gamma:  0.3626274626164527

In [ ]:
# Losses for current model:
# Delta:  0.3170565694387127
# Gamma:  0.46924481339153173
# BEST
# Losses for current model:
# Delta:  0.3740551989443388
# Gamma:  0.4684181799050302
# Losses for current model:
# Delta:  0.18799129964504166
# Gamma:  0.6036873977024664
# BEST
# Losses for current model:
# Delta:  0.0742122219050141
# Gamma:  0.3701475459191158
# BEST